# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,5.53,96,100,6.73,GS,1712506152
1,1,port mathurin,-19.6833,63.4167,27.24,79,45,6.04,MU,1712506152
2,2,yellowknife,62.4560,-114.3525,1.34,91,75,7.20,CA,1712506016
3,3,beloyarskiy,63.7119,66.6722,-3.33,75,100,5.81,RU,1712506153
4,4,port-aux-francais,-49.3500,70.2167,5.24,69,100,6.04,TF,1712506153
...,...,...,...,...,...,...,...,...,...,...
563,563,kharp,66.8014,65.8081,-4.60,95,100,4.51,RU,1712506224
564,564,valparaiso,-33.0393,-71.6273,25.08,60,0,4.12,CL,1712506224
565,565,vorkuta,67.5000,64.0000,-7.16,87,100,6.71,RU,1712506013
566,566,santa maria del rio,21.8000,-100.7500,25.03,21,38,3.42,MX,1712506224


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:

map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700, #optional
    frame_height = 500 #optional
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
filtered_cities = city_data_df[city_data_df['Cloudiness'] == 0
                               & (city_data_df['Max Temp'] <27) & (city_data_df['Max Temp'] >21) & (city_data_df['Wind Speed'] <4)]
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,brak,27.5496,14.2714,29.81,6,0,3.79,LY,1712506153
14,14,la chapelle-saint-mesmin,47.8865,1.8345,19.47,68,0,4.12,FR,1712506154
29,29,al bardiyah,31.7561,25.0865,18.55,59,0,9.00,LY,1712506156
47,47,anadyr,64.7500,177.4833,-20.56,83,0,5.00,RU,1712506158
55,55,fort bragg,35.1390,-79.0060,17.12,40,0,2.57,US,1712505920
...,...,...,...,...,...,...,...,...,...,...
544,544,broome,42.2506,-75.8330,11.21,49,0,5.14,US,1712506221
550,550,nica,43.7031,7.2661,20.53,70,0,3.09,FR,1712506222
551,551,misratah,32.3754,15.0925,21.73,39,0,3.88,LY,1712506222
560,560,khairpur mir's,27.5295,68.7617,29.00,10,0,0.94,PK,1712506223


In [37]:
null_counts = filtered_cities.isnull().sum()
print(null_counts)

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       1
Date          0
dtype: int64


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,brak,LY,27.5496,14.2714,6,
14,la chapelle-saint-mesmin,FR,47.8865,1.8345,68,
29,al bardiyah,LY,31.7561,25.0865,59,
47,anadyr,RU,64.7500,177.4833,83,
55,fort bragg,US,35.1390,-79.0060,40,
...,...,...,...,...,...,...
544,broome,US,42.2506,-75.8330,49,
550,nica,FR,43.7031,7.2661,70,
551,misratah,LY,32.3754,15.0925,39,
560,khairpur mir's,PK,27.5295,68.7617,10,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
#Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    radius = 10000
    params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel",
    "limit": 20
    }

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

brak - nearest hotel: فندق براك السياحي
la chapelle-saint-mesmin - nearest hotel: Formule 1
al bardiyah - nearest hotel: فندق البردي
anadyr - nearest hotel: Гостевой дом
fort bragg - nearest hotel: Airborne Inn Lodging
hamilton - nearest hotel: North Vista Manor
longyearbyen - nearest hotel: The Vault
cefalu - nearest hotel: Madema B&B
tamanrasset - nearest hotel: فندق الأمان
san luis - nearest hotel: Hotel Dos Venados
olovyannaya - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
arar - nearest hotel: دار أوتيل Dar Otel
sirte - nearest hotel: فندق المدينة-سرت
thomson - nearest hotel: Hampton Inn Thomson
colonia - nearest hotel: Wasserturm Hotel Cologne
sur - nearest hotel: Sur Hotel
el-tor - nearest hotel: No hotel found
kolonia - nearest hotel: Wasserturm Hotel Cologne
jamestown - nearest hotel: DoubleTree Jamestown
belyy yar - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
pacific grove - nearest hotel: Pacific Grove Inn
perr

,City,Country,Lat,Lng,Humidity,Hotel Name
8,brak,LY,27.5496,14.2714,6,فندق براك السياحي
14,la chapelle-saint-mesmin,FR,47.8865,1.8345,68,Formule 1
29,al bardiyah,LY,31.7561,25.0865,59,فندق البردي
47,anadyr,RU,64.7500,177.4833,83,Гостевой дом
55,fort bragg,US,35.1390,-79.0060,40,Airborne Inn Lodging
...,...,...,...,...,...,...
544,broome,US,42.2506,-75.8330,49,No hotel found
550,nica,FR,43.7031,7.2661,70,Hôtel du Centre
551,misratah,LY,32.3754,15.0925,39,الفندق السياحى
560,khairpur mir's,PK,27.5295,68.7617,10,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
# Filter hotel_df to exclude rows where "Hotel Name" is "No hotel found"
valid_hotels_df = hotel_df[hotel_df['Hotel Name'] != 'No hotel found']

# Plot the valid hotels using hvplot
plot = valid_hotels_df.hvplot.points(x='Lng', y='Lat', geo=True, color='City', size=100, alpha=0.6, tiles = 'OSM', hover_cols=['City', 'Hotel Name'])

# Show the plot
plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)

In [13]:

map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700, #optional
    frame_height = 500 #optional
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [14]:
valid_hotels_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,brak,LY,27.5496,14.2714,6,فندق براك السياحي
14,la chapelle-saint-mesmin,FR,47.8865,1.8345,68,Formule 1
29,al bardiyah,LY,31.7561,25.0865,59,فندق البردي
47,anadyr,RU,64.7500,177.4833,83,Гостевой дом
55,fort bragg,US,35.1390,-79.0060,40,Airborne Inn Lodging
69,hamilton,US,39.1834,-84.5333,48,North Vista Manor
77,longyearbyen,SJ,78.2186,15.6401,66,The Vault
93,cefalu,IT,38.0386,14.0229,55,Madema B&B
128,tamanrasset,DZ,22.7850,5.5228,8,فندق الأمان
136,san luis,AR,-33.2950,-66.3356,29,Hotel Dos Venados
